# Module 3 - Flow Control (Conditions + Loops)

In previous modules you learned:
- basic data types (`int`, `float`, `bool`, `str`, `None`)
- sequences and collections (`str`, `list`, `tuple`, `dict`, `set`)
- string methods and data wrangling primitives

This module adds the missing superpower: **deciding what runs, and how many times**.

## Learning goals

By the end of this lesson you should be able to:
- write clear conditions with comparison / logical operators
- use `if / elif / else` (including nested conditions) to branch behavior
- use `while` loops for "repeat until..." patterns
- use `for` loops to iterate over sequences/collections and build results
- use `break`, `continue`, and `pass` intentionally (and avoid common pitfalls)

> **Note:** We still avoid *functions* and *error handling* for now - those come next modules.

## From "top-to-bottom" to controlled execution

Python normally executes statements top-to-bottom. Flow control lets us:
- **branch** (choose one path out of many)
- **repeat** (run a block of code multiple times)

In data engineering terms:
- branching is used for validation and routing (e.g., *skip bad records*, *choose parsing strategy*)
- loops are used for batch-like processing (e.g., *process records*, *aggregate metrics*)

## Boolean expressions (quick refresh)

A **boolean expression** evaluates to either `True` or `False`.

Common ingredients:
- comparisons: `== != < <= > >=`
- membership: `in`, `not in`
- identity: `is`, `is not` (mainly with `None`)
- logic: `and`, `or`, `not`

In [ ]:
# Comparison operators

a = 10
b = 3

print(a > b)
print(a == b)
print(a != b)

# Membership operators

word = "Schwarzenegger"
print("egg" in word)
print("zzz" in word)

True
False
True
True
False


### Short-circuit evaluation (important)

Python evaluates `and` / `or` left-to-right and may stop early:
- `X and Y`: if `X` is falsy, Python doesn't need `Y`
- `X or Y`: if `X` is truthy, Python doesn't need `Y`

This matters for performance and for safety (e.g., avoid accessing something that may not exist).

In [ ]:
# Short-circuit examples
x = 0

print(x != 0 and (10 / x) > 1)  # safe: second part is not evaluated
print(x == 0 or (10 / x) > 1)   # safe for same reason

In [ ]:
if x == 0:
  print('error')
elif 10 / x > 1:
  print('good')


if x != 0 and (10 / x) > 1:
  pass

if x is not None and x > 10:
  pass

## Truthiness as a clean guard (quick reference)

We already covered "Pythonic truth" earlier. Here we'll use it **only in context**: writing clean *guards* in real code.

A common data-engineering pattern:

- "Missing / empty" values (`""`, empty containers, `0` in some cases) can be treated as **not present** using `if not x:`
- But when you specifically mean **missing** (not "empty"), prefer an explicit check like `x is None`

> Rule of thumb:  
> - Use `if not x:` when "empty counts as missing"
> - Use `if x is None:` when only `None` means "missing"

Common **falsy** values (treated as `False` in a boolean context): `0`, `''`, `[]`, `None`, `{}`, `False`.

In [1]:
# Falsy: 0, '', [], None, {}, False
print(bool(0), bool(''), bool([]), bool(None))

False False False False


In [ ]:
bool(0)

num = None

num = int()

s = ''
s = 'hello'

if num: # -> if bool(num):
  print(num)

False

In [ ]:
num = 0 # reset

num = input()

if num:
  print(num)
else:
  print('no value in num')

no value in num


In [ ]:
# Clean guard examples (same idea as earlier modules)

raw_user_id = ""      # could come from parsing a line / request payload
raw_amount = None     # could come from JSON

# If empty string is considered "missing", a truthiness guard is clean:
if not raw_user_id:
    print("reject: missing user_id")

# When you mean "missing value" specifically, be explicit:
if raw_amount is None:
    print("reject: missing amount")

in `pandas` we cannot use `if df:`

* to check if a DataFrame is empty `if not df.empty`
* is `df` variable hold something `if df is not None`

### `==` vs `is` (and why `None` is special)

- `==` checks **value equality**
- `is` checks **object identity** (same object in memory)

The most common and recommended use of `is` is comparing to `None`:

```python
if x is None:
    ...
```

In [ ]:
x = None
print(x is None)
print(x == None)  # works, but prefer 'is None'

y = []
z = []
print(y == z)  # same value (both empty lists)
print(y is z)  # different objects

True
True
True
False


In [ ]:
id(z) , id(y)

(133426989769280, 133426989768640)

## `if` statements

**Indentation:** In Python, indentation uses a tab or 4 spaces.

A **code block** is defined after `:` — everything indented under it belongs to that block (`if`, loops, functions).

In [ ]:
# Indentation defines blocks (after :). Use 4 spaces or tab consistently.
if True:
    print("inside block")
    print("still inside")

In [ ]:
if temperature_c > 30:
    # start block
    print("Too hot")
    print('wow')
    print('wow')
    print('wow')

# end block
print('not wow')

IndentationError: unexpected indent (ipython-input-2309004777.py, line 5)

Run a block only when a condition is true:

```python
if temperature_c > 30:
    print("Too hot")
```

### `if` / `else`

Choose between **two** branches. The `else` complements the `if` (anything not falling within the `if` condition):

```python
if is_valid:
    status = "ok"
else:
    status = "reject"
```

**JIT:** Python compiles line by line at runtime.

In this example: if `is_valid` is falsy, we enter the `else` branch.

In [ ]:
# JIT: code is compiled at runtime, line by line
is_valid = 0

if is_valid:
    status = "ok"
else:
    status = "error"   # e.g. skip or log instead of 10/0
print(status)

ZeroDivisionError: division by zero

### `elif` "waterfall" (multi-branch decision)

Use `elif` when you have **multiple mutually exclusive cases**. Python checks top-to-bottom and runs the **first** matching branch, then stops.

```python
if score >= 90:
    grade = "A"
elif score >= 80:
    grade = "B"
else:
    grade = "C or below"
```

In [2]:
score = 'p'
isinstance(score, (int, float))

False

In [ ]:
score = 12.5
# dtype - on us!
# limit - if
# modularty - algo
# 0 / None - on us! if

# always int
if type(score) == int or type(score) is float:
  print('is int')

isinstance(score, (int,float)) # department and sub

if score is None or not isinstance(score, (int,float)) or score > 100 or score < 0 :
  print('Error!')
elif score >= 90:
    grade = "A"
elif score >= 80:
    grade = "B"
else:
    grade = "C or below"

print(grade)

is int
C or below


#### ❗ What to keep in mind

- **Order matters.** Put the *most specific* or *highest priority* tests first.
- Write conditions so they are **mutually exclusive** when possible (less mental load).
- Prefer *clear* conditions over clever ones. (You'll thank yourself later when debugging pipelines.)

### Guards in real data code

A common pattern is to "reject early":

```python
if not record.get("id"):
    status = "reject: missing id"
elif record.get("amount") is None:
    status = "reject: missing amount"
else:
    status = "ok"
```

This reads top-to-bottom like a checklist:

In [ ]:
score = 83

if score >= 90:
    grade = "A"
elif score >= 80:
    grade = "B"
elif score >= 70:
    grade = "C"
elif score >= 60:
    grade = "D"
else:
    grade = "F"

print(grade)

**Pattern:** We don’t control what `input()` returns; we are responsible for validation, the algorithm, and the output.

In [ ]:
# input() → we don't control input; we're responsible for: validation(), algo(), output()

In [ ]:
%%timeit

result = 'F'
score = 60

if score >= 90:
    result = "A"
elif score >= 80:
    result = "B"
elif score >= 70:
    result = "C"
elif score >= 60:
    result = "D"


69.6 ns ± 1.38 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [ ]:
grades ={10:'A' , 9:'B', 8:'C'}

In [ ]:
%%timeit

score = 85
result = grades[score // 10]

51.1 ns ± 0.843 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### Why the order of `elif` tests matters

If a broad condition appears first, it can “steal” cases that should have matched a later, more specific branch.


In [ ]:
value = 15

# WRONG: the first condition is too broad
if value > 0:
    label = "positive"
elif value > 10:
    label = "greater than 10"
else:
    label = "zero or negative"

print("Wrong order ->", label)

# RIGHT: check the more specific condition first
if value > 10:
    label = "greater than 10"
elif value > 0:
    label = "positive"
else:
    label = "zero or negative"

print("Right order  ->", label)

### A realistic "record validation" example

Imagine you read records from a CSV / API and want to validate before loading to a database.

In [ ]:
record = {"name": "Dana", "age": 31, "height_m": 1.73}

# Basic validation / routing
# Using a truthiness guard is a clean way to treat "" as "missing"
if not record.get("name"):
    status = "reject: missing name"
elif record.get("age") is None:
    status = "reject: missing age"
elif record["age"] < 0:
    status = "reject: invalid age"
else:
    status = "ok"

print(status)

ok


### Tip: validating user input with `.isnumeric()`

When reading input with `input()`, the result is always a string. Before converting to `int`, check with `.isnumeric()` to avoid crashes:

In [ ]:
age_str = input("Enter your age: ")

if age_str.strip().isnumeric():
    age = int(age_str)
    print(f"In 10 years you will be {age + 10}")
else:
    print("That was not a valid number.")

### Nested conditions (use them, but don't overdo it)

Nested `if` is sometimes clearer (especially when a later test only makes sense if an earlier test passed).

In [ ]:
text = "The rain in Spain"

if "Spain" in text:
    if "rain" in text.lower():
        print("Weather + location detected")
    else:
        print("Location detected, weather not mentioned")
else:
    print("No location match")

print('hello')

Practically, you could use a boolean expression like this:
```python
if "Spain" in text and "rain" in text.lower():
    ...
```
This is fine if you don't need to know which part of the boolean expression failed (left side or right side of the `and` boolean operator). And in many cases you don't. But if you do, like when you want to take different action in each scenario or log a different message, then you have to go with nested conditions.

In [ ]:
import random

### Q in class

declare new variable grade

set a scroe between 0 to 100 inside grade using `random.random()` function

**use only one print!** to print the student grade in letters:

* 0-50 : fail
* 51-60 : D
* 61- 70: C
* 71-80 : B
* 81-90 : A
* 91-100 : A+

Same pattern: after `input()` — validate, process, then output.

In [ ]:
# Same pattern: input → validate → process → output

In [ ]:
#@title Solution

import random as rnd

start, stop = 0, 100
grade = 'F'
score = int(rnd.random() * 100)
score = rnd.randint(start, stop)

score = None


if score is None:
  raise ValueError('must be a value!')
elif not isinstance(score, int):
  raise ValueError(f'must be a int! got {type(score)}')
elif score > 100 or score < 0:
  raise ValueError(f'grade not in range! must between{start, stop} got:{score}')

if score >= 91:
  grade = 'A+'
elif score >= 81:
  grade = 'A'
elif score >= 71:
  grade = 'B'

print(f'your score {score} is : {grade}')

ValueError: must be a value!

In [ ]:
def main():
  user_score = input()
  if validate_input(user_score):
    check_grade(user_score)
  else:
    error_handling()




def check_grade(score):
  if score >= 91:
  grade = 'A+'
  elif score >= 81:
    grade = 'A'
  elif score >= 71:
    grade = 'B'


### [Optional] Chained comparisons

Python supports math-like chains:

```python
18 <= age < 65
```

This is the same as:

```python
(age >= 18) and (age < 65)
```

In [ ]:
age = 42
print(18 <= age < 65)

### [Optional] Inline `if` (ternary conditional operator)

There is a shortened, single statement way to code an `if`/`else` condition. Nothing that you cannot accomplish with the syntax presented above, just a convenience:
```python
status = "WIP" if len(tasks_list) > 0 else "DONE"
```

In [ ]:
tasks_list = ["do-this", "do-that", "do-da-do-da"]
status = "WIP" if len(tasks_list) > 0 else "DONE"
print(status)

**Difference:** `while` runs as long as the condition is true (we may not know the number of steps in advance). `for` iterates over a known sequence (e.g. list, `range`).

In [ ]:
IS_PROD = False

env_run = 'prod' if IS_PROD else 'qa'


env_run = 'qa'
if IS_PROD:
  env_run = 'prod'


env_run

output_folder_path = input()
if not output_folder_path:
  output_folder_path = create_output_folder()


output_folder_path = output_folder_path if output_folder_path else create_output_folder()


NameError: name 'env_run' is not defined

In [ ]:
# while: repeat while condition holds (unknown number of steps)
# for: iterate over a sequence (known end)

## `while` loops

In [ ]:
n = -3

while n < 0:
  print('hello')
  n -= 1
  if n :
    break

Use `while` when you want to repeat **until a condition becomes false**.

Typical pattern:
- initialize state
- loop while condition holds
- update state each iteration

> **Important**: Watch out for infinite loops: if you forget to update the state, the condition may never change.

In [ ]:
# Sum numbers from 1..n using a while-loop
n = 5

total = 0
i = 1
while i <= n:
    total = total + i
    i = i + 1

print(total)

### `break` inside `while`

Sometimes the "stop rule" is naturally expressed using `break`.

This can be clearer than writing a complex loop condition.

In [ ]:
# Find the first power of 2 that is >= target

target = 70

value = 1
while True:
    if value >= target:
        break
    value = value * 2

print(value)

In [ ]:
target = 20
value = 0
while value <= target:
    value += 1
    if value % 2 != 0 :
      continue
    print(value)

print(value)

2
4
6
8
10
12
14
16
18
20
21


### Exercise: Pincode Guessing Game

A classic `while True` + `break` pattern. The loop runs indefinitely until the user guesses correctly.

In [ ]:
pincode = 1337

while True:
    attempt = int(input("Enter the pin code: "))
    if attempt == pincode:
        print("You got it!")
        break
    elif attempt < pincode:
        print("Too low, try again")
    else:
        print("Too high, try again")

## `for` loops

Use `for` when you want to iterate over an **iterable** (something that produces items one at a time).

You can iterate over:
- sequences: `str`, `list`, `tuple`
- sets (unordered)
- dicts (by default iterates keys)

In [ ]:
text = "data"
# decalred inside the for loop
# assigned value on each iter
for ch in text:
    print(ch)


# alive also after the loop
print(ch)

d
a
t
a
a


In [ ]:
colors = ["red", "green", "blue"]
for c in colors:
    print(c)

red
green
blue


In [ ]:
person = {"name": "Noa", "age": 27}
for key in person:
    print(key, person[key])

name Noa
age 27


## Q in class

iterate over this list

 `L = [1,2,3,4,5,6,7,8,9,10]`

 save all the even and odd numbers each on a seperate list

In [ ]:
L = [1,2,3,4,5,6,7,8]

d = {'even':[] , 'odd':[]}

for num in L :
  if num % 2 == 0 :
    d['even'].append(num)
  else:
    d['odd'].append(num)


d = {'even':[] , 'odd':[]}

for num in L :
  key = 'even' if num % 2 == 0 else 'odd'
  d[key].append(num)


print(d)

{'even': [2, 4, 6, 8], 'odd': [1, 3, 5, 7]}


### Unpacking items while iterating

Often you iterate over rows returned from a database driver. A row is commonly represented as a **tuple**.

When each row has a fixed "shape", you can **unpack** it directly in the `for` statement:

```python
for name, age, height, user_id in db_result_set:
    ...
```

This is the same tuple-unpacking idea you saw earlier - just applied during iteration.

In [ ]:
# Simulated DB result set: each row is a tuple
db_result_set = [
    ("Jack", 30, 1.78, "U001"),
    ("Maya", 27, 1.62, "U002"),
    ("Omer", 41, 1.83, "U003"),
]

for name, age, height, user_id in db_result_set:
    # In real pipelines you might validate, transform, and load
    print(f"{user_id} -> {name} ({age}y, {height}m)")

In [ ]:
result_dict.items()

dict_items([('a', 1), ('b', 2), ('c', 3)])

In [ ]:
result_dict = {'user':'onn',
               'data':[{'date':11010,'age':15}],
               'tsmp':'aaa'}

for key, value in result_dict.items():
  print(key , value)


for item in result_dict.items():
  key , value = item
  print(key , value)


# for key in result_dict:
#   value = result_dict[key]
#   print(key, value)



# result_dict = {'user':'onn',
#                'data':[{'date':11010,'age':15}],
#                'tsmp':date.date()}


# for user_name, data, tsmp in result_dict:
#   data = ....

user onn
data [{'date': 11010, 'age': 15}]
tsmp aaa


In [ ]:
L = [1,2,3]

for i in L :
  i = 5

print(L)

### `range()` for counting loops

`range(stop)` produces `0...stop-1`  
`range(start, stop)` produces `start...stop-1`  
`range(start, stop, step)` supports skipping.

In [ ]:
for i in range(5):
    print(i)

0
1
2
3
4


In [ ]:
for i in range(3, 8):
    print(i)

3
4
5
6
7


In [ ]:
for i in range(10, 0, -2):
    print(i)

10
8
6
4
2


In [ ]:
list(range(10,0,-1))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [ ]:
L = [1,2,3,4,5,6]

# range(len(L)) == range(0,6) ==> range(6) ==> 0,1,2,3,4,5
for i in range(len(L)):
  L[i] *= 10


L

[10, 20, 30, 40, 50, 60]

## Q in class

get user input for `start` and `end`

write a FOR loop using `range()` staring from `start` to `end` including the end!

add to list `odd_list` all the odd numbers

add to list `even_list` all the odd numbers

````
input 0, 5

output:
odd_list: [1,3,5]
even_list: [0,2,4]

````

## STOP HERE @@

In [ ]:
#@title Solution

start, end = 2, 10

odd_list, even_list = [], []

for i in range(start, end + 1):
  if i % 2 == 0:
    even_list += [i] # even_list = even_list + [i]
  else:
    odd_list.append(i)

print(even_list, odd_list, sep='\n')

[0, 2, 4]
[1, 3, 5]


In [ ]:
L = [10,20,30,40,50]

# 0,1,2,3,4
for i in range(len(L)):
  L[i] *= 10

L

# for -> forEach , run all values -> for iter -> run all values obj (str, tuple, list, dict, range)
# for index -> range() -> run all values range | manipulate mtuable object


TypeError: 'int' object is not iterable

## Q in class

loop over this list

` L = ['Python','SQL','PBI','Onn']`

the print each word and then each of its letter seperate by `.`

just use `for loop` and `print()`

```
=== Python ===
P.y.t.h.o.n.
=== SQL ===
S.Q.L.
=== PBI ===
P.B.I.
=== Onn ===
O.n.n.
```

In [ ]:
print(*args, sep=' ', end='\n', file=None, flush=False)

In [ ]:
L = ['Python','SQL','PBI','Onn']

#option 1
for word in L :
  print(f'{'='*3} {word:^5} {'='*3}')
  for c in word:
    print(c, end='.')
  print()

#option 2
for word in L :
  print(f'{'='*3} {word:^5} {'='*3}')
  tmp_result = '' # reset variable
  for c in word:
    tmp_result += f'{c}.'
  print(tmp_result) # finishing move

#option 3
for word in L :
  print(f'{'='*3} {word:^5} {'='*3}\n{'.'.join(word)}')

=== Python ===
P.y.t.h.o.n.
===  SQL  ===
S.Q.L.
===  PBI  ===
P.B.I.
===  Onn  ===
O.n.n.
=== Python ===
P.y.t.h.o.n.
===  SQL  ===
S.Q.L.
===  PBI  ===
P.B.I.
===  Onn  ===
O.n.n.
=== Python ===
P.y.t.h.o.n
===  SQL  ===
S.Q.L
===  PBI  ===
P.B.I
===  Onn  ===
O.n.n


**Example:** Sum each sublist and print (or store in a new list like `[6, 0, 11]`). Note: reset `subtotal` inside the outer loop.

In [ ]:
sales = [[1,2,3], [0,0], [5,6]]

# Goal: sum each sublist (e.g. [6, 0, 11]); total = 0
total = 0

for sublist in sales:
  for value in sublist:
    total += value
  print(total)

print()

total = 0

for sublist in sales:
  subtotal = 0 # reset
  for value in sublist:
    subtotal += value
  print(subtotal)
  total += subtotal # finish
print(total)


6
6
17

6
0
11
17


### `enumerate()` for index + value

When you need both position and item, prefer `enumerate()` over manual counters.

In [ ]:
animals = ["cat", "dog", "owl"]

for idx, animal in enumerate(animals):
    print(idx, animal)

0 cat
1 dog
2 owl


In [ ]:
list(enumerate(animals))

[(0, 'cat'), (1, 'dog'), (2, 'owl')]

In [ ]:
#@title check headers alignment example
# Validating headers in a raw CSV file before transformation
expected_columns = ['user_id', 'transaction_date', 'amount', 'currency']
raw_headers = ['user_id', 'txn_date', 'amt', 'curr'] # Slightly mismatched headers

for i, col_name in enumerate(expected_columns):
    if raw_headers[i] != col_name:
        # Identifying the exact column position that broke the schema contract
        print(f"Schema Mismatch: Column {i} ('{raw_headers[i]}') does not match expected '{col_name}'")
    else:
        print(f"Column {i} ('{col_name}') validated.")

Column 0 ('user_id') validated.
Schema Mismatch: Column 1 ('txn_date') does not match expected 'transaction_date'
Schema Mismatch: Column 2 ('amt') does not match expected 'amount'
Schema Mismatch: Column 3 ('curr') does not match expected 'currency'


### Practical example: `enumerate` + `continue` + `break`

Combine `enumerate()` with loop control to build a filtered list. Here we skip certain names and stop early on a condition.

In [ ]:
first_names = ["Shifra", "David", "Noa", "Sivan", "Livnat"]
last_names  = ["Levi", "Cohen", "Dan", "Nir", "Liran"]

full_name_list = []

for idx, name in enumerate(first_names):
    if name.startswith("Sh"):
        continue

    if name.endswith("n"):
        print(f"Breaking at: {name}")
        break

    full_name = f"{name} {last_names[idx]}"
    full_name_list.append(full_name)

print(full_name_list)

In [ ]:
#@title [optional] example Enum class
from enum import Enum, auto

class PipelineStatus(Enum):
    PENDING = auto()    # Automatically assigns 1
    RUNNING = auto()    # Automatically assigns 2
    COMPLETED = auto()  # Automatically assigns 3
    FAILED = auto()     # Automatically assigns 4


# class PipelineStatus(Enum):
#     PENDING = 0
#     RUNNING = 1
#     COMPLETED = 2
#     FAILED = 3

current_status = PipelineStatus.RUNNING

print(current_status.value)

if current_status == PipelineStatus.FAILED:
    print("Triggering alert...")

2


In [ ]:
#@title [optional] advance example Enum class
from enum import Enum

class UserRole(Enum):
    # We define the members as (value, permissions)
    ADMIN = ("admin", ["create", "edit", "delete"])
    EDITOR = ("editor", ["edit"])
    VIEWER = ("viewer", [])

    def __init__(self, val, permissions):
        self._value_ = val
        self.permissions = permissions

# Usage
print(UserRole.ADMIN.value)       # "admin"
print(UserRole.ADMIN.permissions) # ['create', 'edit', 'delete']

admin
['create', 'edit', 'delete']


### `zip()` for parallel iteration

Use `zip()` to iterate multiple sequences together (pairs/tuples of items).

In [ ]:
names = ["Avi", "Maya", "Lior"]
scores = [90, 78, 88]

for name, score in zip(names, scores):
    print(f"{name}: {score}")

Avi: 90
Maya: 78
Lior: 88


In [ ]:
names = ["Avi", "Maya", "Lior"]
scores = [90, 78, 88]
age = [12,15,16]

list(zip(names, scores,age))

[('Avi', 90, 12), ('Maya', 78, 15), ('Lior', 88, 16)]

## List Comprehensions

Now that you know `for` loops, you can learn a concise "one-line" syntax for creating lists: the **list comprehension**.

```python
[expression for item in iterable]
[expression for item in iterable if condition]
```

This replaces the common pattern of "create empty list, loop, append." The result is shorter, more readable, and often faster.

In [ ]:
# Traditional way with a for loop
squares_loop = []
for n in range(10):
    squares_loop.append(n ** 2)
print("With loop:", squares_loop)

# Same result using a list comprehension
squares_comp = [n ** 2 for n in range(10)]
print("With comprehension:", squares_comp)

### List comprehension with a condition

Add `if` at the end to filter items. Only items where the condition is `True` make it into the result.

In [ ]:
# Filter: only keep planets with short names
planets = ["Mercury", "Venus", "Earth", "Mars", "Jupiter"]
short_planets = [planet for planet in planets if len(planet) < 6]
print("Short planets:", short_planets)

# Data engineering example: clean a list of IDs
raw_ids = ["A1", "", "B2", None, "C3", ""]
clean_ids = [uid for uid in raw_ids if uid]
print("Clean IDs:", clean_ids)

## Dictionary Comprehensions

The same idea works for dictionaries. The syntax uses `{key: value for ...}`:

```python
{key_expr: value_expr for item in iterable}
{key_expr: value_expr for item in iterable if condition}
```

Useful for transforming or filtering dictionaries in one line.

In [ ]:
products = {"AG32": 10, "HT91": 12, "PL65": 30, "OS31": 15}

# Filter: keep only products with price > 15
top_products = {pid: price for pid, price in products.items() if price > 15}
print("Top products:", top_products)

# Transform: build a name -> name_length mapping
names = ["Avi", "Maya", "Lior"]
name_lengths = {name: len(name) for name in names}
print("Name lengths:", name_lengths)

## Loop control statements

- `break`: exit the loop immediately
- `continue`: skip to the next iteration
- `pass`: do nothing (placeholder, useful while building code)

In [ ]:
# Process integers, skipping negatives exiting when zero
values = [3, -1, 7, 0, 4]

for v in values:
    if v < 0:
        print("Skipping negative:", v)
        continue
    if v == 0:
        print("Found zero, stopping")
        break
    print("Processed:", v)


Processed: 3
Skipping negative: -1
Processed: 7
Found zero, stopping


In [ ]:
# Process integers, skipping negatives exiting when zero
values = [3, -1, 7, 0, 4]
#   [(0,3)..(4,4)]

for idx, v in enumerate(values):
    if v < 0:
        print("Skipping negative:", v ,f'idx: {idx}')
        continue
    if v == 0:
        print("Found zero, stopping",f'idx: {idx}')
        break
    print("Processed:", v ,'in' ,f'idx: {idx}')

Processed: 3 in idx: 0
Skipping negative: -1 idx: 1
Processed: 7 in idx: 2
Found zero, stopping idx: 3


## Mini "pipeline-style" example: count bad records

Goal: given a list of dicts, count how many records are valid.

In [ ]:
records[2].get('id', None)

''

In [ ]:
records = [
    {"amount": -1},
    {"id": "A1", "amount": 10.5},
    {"id": "A2", "amount": None},
    {"id": "",   "amount": 7.0},
    {"id": "A3", "amount": -1},
]

valid_count = 0
invalid_count = 0

for r in records:
    if not r.get("id",None): # None / ''
    #if not r['id']:
        invalid_count += 1
        continue
    if r.get("amount") is None or r["amount"] < 0:
        invalid_count += 1
        continue
    valid_count += 1

print("valid:", valid_count)
print("invalid:", invalid_count)

valid: 1
invalid: 4


In [ ]:
numbers = [0,1,0,2,-15,0,7,8,-100000]

# only total positive numbers
# split even and odd to different lists

even, odd = [], []

for num in numbers:
  if num <= 0:
    continue

  if num % 2 == 0 :
    even.append(num)
  else:
    odd.append(num)

even, odd

([2, 8], [1, 7])

In [ ]:
expected_columns = ['user_id', 'transaction_date', 'amount', 'currency']
#raw_headers = ['user_id', 'txn_date', 'amt', 'curr'] # Slightly mismatched headers
raw_headers = ['user_id', 'transaction_date', 'amount', 'currency']

import numpy as np

np.all(np.equal(expected_columns, raw_headers))

np.True_

In [ ]:
%%timeit

for i in range(len(expected_columns)):
  if expected_columns[i] != raw_headers[i]:
    break

202 ns ± 52.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
%%timeit
np.all(np.equal(expected_columns, raw_headers))

11.3 µs ± 5.48 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
with open('my_data.csv','w') as f:
  print(csv_data, file=f)

In [ ]:
csv_data = """
id,name,age
1,2,3
1,2,3,4
1,2,3
"""

In [ ]:
error_list

[['1', '2', '3', '4']]

In [ ]:
error_list = []

def bad_rows(row):
  error_list.append(row)
  return None


pd.read_csv('my_data.csv' ,
            on_bad_lines=bad_rows,
            engine='python')

,id,name,age
0,1,2,3
1,1,2,3


In [ ]:
csv_data = "id,name,age\n1,2,3\n1,2,3,4\n1,2,3"

import pandas as pd


pd.read_csv(csv_data)

FileNotFoundError: [Errno 2] No such file or directory: 'id,name,age\n1,2,3\n1,2,3,4\n1,2,3'

## Exercises

> Run the "Setup" cell under each exercise section first.

### Exercise 1 - Largest of three numbers (if)

Given 3 numbers `a`, `b`, `c`, set `largest` to the largest value and print it.

In [ ]:
#Setup
a = 7
b = 12
c = 9

In [ ]:
# your code here


#### Solution

In [ ]:
largest = a
if b > largest:
    largest = b
if c > largest:
    largest = c

print(largest)

In [ ]:
a = 7
b = 12
c = 9

#len, max, min, sum

max([a,b,c])

12

### Exercise 2 - Sentence length classifier (if + strings)

Split a sentence into words and classify:
- 4 words or less → `Short sentence`
- 5-10 words → `Average sentence`
- 11+ words → `Long sentence`

In [ ]:
# Setup
sentence = "Python is fun"

In [ ]:
# your code here


#### Solution

In [ ]:
words = sentence.split()
count = len(words)

if count <= 4:
    print("Short sentence")
elif count <= 10:
    print("Average sentence")
else:
    print("Long sentence")

### Exercise 3 - Sum a range (while)

Compute the sum of integers from `start` to `end` (inclusive) using a `while` loop.

In [ ]:
# Setup
start = 3
end = 7

In [ ]:
# your code here


#### Solution

In [ ]:
total = 0
i = start
while i <= end:
    total += i
    i += 1

print(total)

### Exercise 4 - Divisors (for)

Print all divisors of `n` (numbers that divide `n` with no remainder).

In [ ]:
# Setup
n = 12

In [ ]:
# your code here


#### Solution

In [ ]:
for d in range(1, n + 1):
    if n % d == 0:
        print(d)

### Exercise 5 - Multiplication table (nested loops, optional challenge)

Print a 1 - 10 multiplication table.
Don't worry about perfect alignment - correctness first.
(If you want alignment, try f-strings with width like `f"{value:4d}"`.)

In [ ]:
# your code here


#### Solution

In [ ]:
for row in range(1, 11):
    line = ""
    for col in range(1, 11):
        value = row * col
        line += f"{value:4d}"
    print(line)

### Exercise 5b - Find the "secret" (nested loops + flag pattern)

Given a list of lists (a "table"), find which row and column contain the word `"secret"`.

**Hint**: Use a boolean flag to break out of nested loops. The inner `break` only exits the inner loop, so you need a second check to exit the outer loop.

In [ ]:
# Setup
table = [
    ["apple", "orange", "grapes", "banana"],
    ["big", "small", "secret"],
    ["red", "black", "blue", "green", "white"],
]

In [ ]:
# Your code here


#### Solution

In [ ]:
found = False

for row_idx, row in enumerate(table):
    for col_idx, item in enumerate(row):
        if item == "secret":
            print(f"Found 'secret' at row {row_idx}, column {col_idx}")
            found = True
            break
    if found:
        break

if not found:
    print("'secret' not found in the table")

### Exercise 6 [challenging] - Longest numbers streak

Given a list of numbers, print the longest **streak** length (which number appears continuously the longest in the list).

For example, given the list of numbers below, then number would be `9` and the streak length would be `4`.

In [ ]:
# Setup
numbers = [2, 9, 11, 3, 3, 7, 9, 9, 9, 9, 5, 5, 5, 11]

In [ ]:
# Your code here


#### Solution

In [ ]:
streak_counter = max_streak_length = 0
streak_n = max_streak_n = None

for n in numbers:
    # Detect when a streak ends
    if n != streak_n:
        # Update the max streak thus far
        if streak_counter > max_streak_length:
            max_streak_length = streak_counter
            max_streak_n = streak_n
        # Initialize a new streak
        streak_n = n
        streak_counter = 1
    else:
        streak_counter += 1

print(f"The number {max_streak_n} appears {max_streak_length} times in a row in the list of numbers.")

<hr/>